In [3]:
from util.utils import evaluate_imgs

In [4]:
vi_img_folder = "/home/2021/hexin/XJ2/D3FM/input2/vi"
ir_img_folder = "/home/2021/hexin/XJ2/D3FM/input2/ir"
f_img_folder = "/home/2021/hexin/XJ2/D3FM/output2/recon"
evaluate_imgs(f_img_folder,vi_img_folder,ir_img_folder,"RGB")






--------------------FLIR_00006.jpg--------------------
SSIM_1: 0.718248405506776, SSIM_2: 0.8200912295601457
Qabf1: 0.0, Qabf2: 0.0
VIF1: 0.0, VIF2: 0.0
MI1: 0.2631033340475172,MI2: 0.8474935472821341
SD: 0.12314768880605698
EN: 6.948622444678215
--------------------FLIR_03952.jpg--------------------
SSIM_1: 0.7950431910951663, SSIM_2: 0.8560124611247301
Qabf1: 0.0, Qabf2: 0.0
VIF1: 0.0, VIF2: 0.0
MI1: 0.818661289729866,MI2: 1.0767099748727131
SD: 0.1835000365972519
EN: 7.110542725918437
--------------------FLIR_08248.jpg--------------------
SSIM_1: 0.8396374801712024, SSIM_2: 0.7490706938619035
Qabf1: 0.0, Qabf2: 0.0
VIF1: 0.0, VIF2: 0.0
MI1: 0.5923858401716775,MI2: 0.7240393230060445
SD: 0.1377478539943695
EN: 6.918008135507429
